In [1]:
import pandas as pd

In [2]:
bad_actor_list=pd.read_csv('bad_actors_list_v1.csv',index_col=['CUSTOMER_ID'])
bad_actor_index=bad_actor_list.index

In [3]:
data=pd.read_csv('clean_uoft_nodes_v2.csv',parse_dates=['BIRTH_DT','CUST_ADD_DT'],index_col=['CUSTOMER_ID'])

In [4]:
col=['CASH_CNT_IN','CASH_SUM_OUT','CASH_SUM_IN','CASH_CNT_OUT', 'WIRES_SUM_IN', 'WIRES_CNT_IN',
       'WIRES_SUM_OUT', 'WIRES_CNT_OUT']
# ## Some the features have 0 and give inf or Nan when we divide
for i in col:
  data[i]=data[i]+1

In [5]:
data['cash_diff']=data['CASH_SUM_IN']-data['CASH_SUM_OUT']
data['wire_diff']=data['WIRES_SUM_IN']-data['WIRES_SUM_OUT']
data['cash_cnt_diff']=data['CASH_CNT_IN']-data['CASH_CNT_OUT']
data['wire_cnt_diff']=data['WIRES_CNT_IN']-data['WIRES_CNT_OUT']
data['avg_cash_transaction_out']=data['CASH_SUM_OUT']/data['CASH_CNT_OUT']
data['avg_wire_transaction_out']=data['WIRES_SUM_OUT']/data['WIRES_CNT_OUT']
data['avg_cash_transaction_in']=data['CASH_SUM_IN']/data['CASH_CNT_IN']
data['avg_wire_transaction_in']=data['WIRES_SUM_IN']/data['WIRES_CNT_IN']
data['monthly_cash_in']=data['CASH_SUM_IN']/12
data['monthly_cash_out']=data['CASH_SUM_OUT']/12
data['monthly_wire_in']=data['WIRES_SUM_IN']/12
data['monthly_wire_out']=data['WIRES_SUM_OUT']/12
data['year_diff']=data['CUST_ADD_DT'].dt.year-data['BIRTH_DT'].dt.year

In [6]:
data.drop(['BIRTH_DT', 'CUST_ADD_DT','NAME'],inplace=True,axis=1)
data=pd.get_dummies(data,columns=['OCPTN_NM','COUNTRY_RISK_INCOME','COUNTRY_RISK_RESIDENCY','GENDER'])

In [7]:
data.head()

,RES_CNTRY_CA,CNTRY_OF_INCOME_CA,PEP_FL,CASH_SUM_IN,CASH_CNT_IN,CASH_SUM_OUT,CASH_CNT_OUT,WIRES_SUM_IN,WIRES_CNT_IN,WIRES_SUM_OUT,...,OCPTN_NM_Low,OCPTN_NM_Moderate,COUNTRY_RISK_INCOME_High,COUNTRY_RISK_INCOME_Low,COUNTRY_RISK_INCOME_Moderate,COUNTRY_RISK_RESIDENCY_High,COUNTRY_RISK_RESIDENCY_Low,COUNTRY_RISK_RESIDENCY_Moderate,GENDER_Female,GENDER_Male
CUSTOMER_ID,,,,,,,,,,,,,,,,,,,,,
935382,1,1,0,2578.785,3,2852.663,8,70775,63,24245,...,0,0,0,1,0,0,1,0,1,0
2305,1,1,0,3036.502,3,4806.997,16,282930,77,196440,...,0,0,0,1,0,0,1,0,0,1
472403,1,1,0,1618.571,2,3483.809,12,7633,7,33599,...,0,0,0,1,0,0,1,0,0,1
227994,1,1,0,3588.042,6,1941.943,9,149254,72,105897,...,0,0,0,1,0,0,1,0,1,0
418265,1,1,0,1726.524,3,13198.169,44,92319,97,187404,...,0,0,0,1,0,0,1,0,1,0


## Create Label for bad actors 

In [8]:
data['label']=None

In [9]:
data.loc[bad_actor_index,'label']=1

In [10]:
data['label']=data['label'].fillna(0)

## check our distribution it should be 50 counts of 1 and 999950 counts of 0

In [11]:
data['label'].value_counts()

0    999950
1        50
Name: label, dtype: int64

## Sampling data. 
Since we are only having two classes, now it important how we sample the data since now both low+medium are in a single class. 
Main idea is take 100 samples each from low and medium so that we ensure some of the actual medium samples are also counted into the data. This would allow the model to identify the medium samples are low. Which otherwise might not be true if we just do random sampling.

Use 25 samples of high risk to oversample to bring 100 samples of each class. Using stratify split to have 50 of each class in train test. 

Mainly because oversampling data to match the actual low samples is not viable and will just introduce more noise. 

In [12]:
medium_data=data[data['RISK']=='medium']
medium_index=medium_data.index
low_data=data[data['RISK']=='low']
low_index=low_data.index

In [13]:
df_1=data[data['label']==1]
df_1_idx=df_1.index

Now check if index values are present in low and medium classes if yes, we need to remove those samples to avoid duplicates when random sampling or we can do that after concat alsow.

In [14]:
df_1.index.isin(low_data.index)

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False])

no index is matching from low

In [15]:
df_1.index.isin(medium_data.index)

array([False, False, False, False, False, False, False, False, False,
        True, False, False, False, False,  True, False,  True,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False])

4 indexes are matching from medium. We can delete duplicates after merging. 

In [16]:
df_2=medium_data.sample(50)
df_3=low_data.sample(50)

In [17]:
df_final=pd.concat([df_1,df_2,df_3])

In [18]:
df_final.columns

Index(['RES_CNTRY_CA', 'CNTRY_OF_INCOME_CA', 'PEP_FL', 'CASH_SUM_IN',
       'CASH_CNT_IN', 'CASH_SUM_OUT', 'CASH_CNT_OUT', 'WIRES_SUM_IN',
       'WIRES_CNT_IN', 'WIRES_SUM_OUT', 'WIRES_CNT_OUT', 'RISK', 'cash_diff',
       'wire_diff', 'cash_cnt_diff', 'wire_cnt_diff',
       'avg_cash_transaction_out', 'avg_wire_transaction_out',
       'avg_cash_transaction_in', 'avg_wire_transaction_in', 'monthly_cash_in',
       'monthly_cash_out', 'monthly_wire_in', 'monthly_wire_out', 'year_diff',
       'OCPTN_NM_High', 'OCPTN_NM_Low', 'OCPTN_NM_Moderate',
       'COUNTRY_RISK_INCOME_High', 'COUNTRY_RISK_INCOME_Low',
       'COUNTRY_RISK_INCOME_Moderate', 'COUNTRY_RISK_RESIDENCY_High',
       'COUNTRY_RISK_RESIDENCY_Low', 'COUNTRY_RISK_RESIDENCY_Moderate',
       'GENDER_Female', 'GENDER_Male', 'label'],
      dtype='object')

In [19]:
df_final.drop(['RISK'],inplace=True,axis=1)

In [20]:
df_final

,RES_CNTRY_CA,CNTRY_OF_INCOME_CA,PEP_FL,CASH_SUM_IN,CASH_CNT_IN,CASH_SUM_OUT,CASH_CNT_OUT,WIRES_SUM_IN,WIRES_CNT_IN,WIRES_SUM_OUT,...,OCPTN_NM_Moderate,COUNTRY_RISK_INCOME_High,COUNTRY_RISK_INCOME_Low,COUNTRY_RISK_INCOME_Moderate,COUNTRY_RISK_RESIDENCY_High,COUNTRY_RISK_RESIDENCY_Low,COUNTRY_RISK_RESIDENCY_Moderate,GENDER_Female,GENDER_Male,label
CUSTOMER_ID,,,,,,,,,,,,,,,,,,,,,
214538,1,1,0,25832.652,12,8489.000,3,6683717,173,3287716,...,0,0,1,0,0,1,0,1,0,1
917730,1,1,0,2409.782,2,27873.000,33,796376,36,218560,...,0,0,1,0,0,1,0,0,1,1
359412,1,1,0,60349.480,32,30092.000,31,7049311,153,3055464,...,0,0,1,0,0,1,0,1,0,1
763368,1,1,0,17930.276,9,315.000,2,75442,19,17400,...,0,0,1,0,0,1,0,1,0,1
404022,1,1,0,42625.993,24,55846.000,37,6000240,122,2485034,...,0,0,1,0,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990297,1,1,0,723.912,2,6492.684,24,46681,41,24178,...,0,0,1,0,0,1,0,1,0,0
743932,1,1,0,2254.559,3,5684.855,22,166923,89,282586,...,0,0,1,0,0,1,0,1,0,0
261459,1,1,0,1408.234,4,11674.889,44,186287,124,252523,...,0,0,1,0,0,1,0,0,1,0


## Check if the merged df has duplicates, if yes then delete else proceed forward.

In [21]:
if df_final.index.has_duplicates==True:
  df_final.drop_duplicates()
  print("Duplicates found and removed")
else:
  print("No duplicates found")

No duplicates found


## Split into train_test

In [22]:
y=df_final['label']
x=df_final.loc[:,df_final.columns!='label']

In [23]:
print(x.shape,y.shape)

(150, 35) (150,)


In [24]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,stratify=y)

In [25]:
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

(105, 35) (105,) (45, 35) (45,)


In [26]:
y_train.value_counts()

0    70
1    35
Name: label, dtype: int64

In [27]:
y_test.value_counts()

0    30
1    15
Name: label, dtype: int64

## Standarize for linear models like SVM etc

In [28]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(x_train)
x_train_std=scaler.transform(x_train)
x_test_std=scaler.transform(x_test)

In [29]:
x_train_std.shape

(105, 35)

In [30]:
x_test_std.shape

(45, 35)

## try oversampling to bring traint to equal distribution

In [31]:
from imblearn.over_sampling import SMOTE
sm = SMOTE()
X_res, y_res = sm.fit_resample(x_train_std, y_train)
print(X_res.shape,y_res.shape)
print(y_res.value_counts())

(140, 35) (140,)
0    70
1    70
Name: label, dtype: int64


In [32]:
from sklearn.metrics import *
from sklearn.ensemble import *
import xgboost as xgb
model_1=RandomForestClassifier()
model_2=HistGradientBoostingClassifier()
model_3=xgb.XGBClassifier()
model_1.fit(X_res,y_res)
model_2.fit(X_res,y_res)
model_3.fit(X_res,y_res)

XGBClassifier()

In [33]:
def metrics(y_true,y_pred):
  print(classification_report(y_true,y_pred))
  print(confusion_matrix(y_true,y_pred))

In [34]:
y_pred_1=model_1.predict(x_test_std)
y_pred_2=model_2.predict(x_test_std)
y_pred_3=model_3.predict(x_test_std)

In [35]:
metrics(y_test,y_pred_1)

              precision    recall  f1-score   support

           0       0.90      0.87      0.88        30
           1       0.75      0.80      0.77        15

    accuracy                           0.84        45
   macro avg       0.82      0.83      0.83        45
weighted avg       0.85      0.84      0.85        45

[[26  4]
 [ 3 12]]


In [36]:
metrics(y_test,y_pred_2)

              precision    recall  f1-score   support

           0       0.89      0.83      0.86        30
           1       0.71      0.80      0.75        15

    accuracy                           0.82        45
   macro avg       0.80      0.82      0.81        45
weighted avg       0.83      0.82      0.82        45

[[25  5]
 [ 3 12]]


In [37]:
metrics(y_test,y_pred_3)

              precision    recall  f1-score   support

           0       0.96      0.83      0.89        30
           1       0.74      0.93      0.82        15

    accuracy                           0.87        45
   macro avg       0.85      0.88      0.86        45
weighted avg       0.89      0.87      0.87        45

[[25  5]
 [ 1 14]]


In [38]:
print(accuracy_score(y_res,model_1.predict(X_res)))

1.0


## Try some linear classification models like SVM , Logisitic Regression etc

In [39]:
from sklearn.svm import LinearSVC
svc = LinearSVC()
svc.fit(X_res,y_res)
y_pred_4=svc.predict(x_test_std)
metrics(y_test,y_pred_4)

              precision    recall  f1-score   support

           0       0.93      0.83      0.88        30
           1       0.72      0.87      0.79        15

    accuracy                           0.84        45
   macro avg       0.82      0.85      0.83        45
weighted avg       0.86      0.84      0.85        45

[[25  5]
 [ 2 13]]


/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [40]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(X_res,y_res)
y_pred_5=lr.predict(x_test_std)
metrics(y_test,y_pred_5)

              precision    recall  f1-score   support

           0       0.89      0.80      0.84        30
           1       0.67      0.80      0.73        15

    accuracy                           0.80        45
   macro avg       0.78      0.80      0.78        45
weighted avg       0.81      0.80      0.80        45

[[24  6]
 [ 3 12]]
